# TODO

- lav similarity matrix på lsh pakken som er i deepnote 
- repliker lsh pakkens resultater i løsningsscript med tf-idf 
- lav recomendation funktion 
- wordclouds 
- lav notebooken lækker med beskrivelser af metoder


# load data med mrjob

In [91]:
%load_ext autoreload
%autoreload 2
!python -m pip install -r requirements.txt
from utils.initialization import *


# med mrjob
# names, party and twitter id
from Data.twitter_ids import twitter_ids
data = pd.DataFrame(columns=['name', "party", 'twitter_id'])
i = 0
for party in twitter_ids:
    for person in twitter_ids[party]:
        data.loc[i, :] = [person, party, twitter_ids[party][person]]
        i += 1

# tweets
filename = "Data/cleaned_data.csv"
if not os.path.exists(filename):
    os.system(f"python utils/clean_data_mrjob.py Data/tweets > Data/tmp_cleaned_data.txt")
    data_ = pd.DataFrame(columns=["name", "tweets"])
    with open("Data/tmp_cleaned_data.txt", "rb") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            line = eval(line.decode())
            data_.loc[i,"name"] = list(line.keys())[0]
            data_.loc[i, "tweets"] = list(line.values())[0]
    data_.to_csv(filename, index = False)

data_ = pd.read_csv(filename)
data = data.merge(data_)
data.tweets = [eval(t) for t in data.tweets]
data["tokens"] = [[w for w in word_tokenize(" ".join(data["tweets"][i])) if w.isalnum()] for i in range(len(data))]
import simplemma #use simplemma instead of nltk.WordNetLemmatizer()          ### vi har addet denne linje
data["tokens"] = [[simplemma.lemmatize(w, lang='da') for w in data["tokens"][i]] for i in range(len(data))] ### vi har addet denne linje
# tokens = [simplemma.lemmatize(t, lang='da') for t in data["tokens"]] ### vi har addet denne linje

data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


You should consider upgrading via the 'c:\Users\bayka\Anaconda3\envs\katrine_personal_env\python.exe -m pip install --upgrade pip' command.


,name,party,twitter_id,tweets,tokens
0,magnus_heunicke,socialdemokratiet,22695562,[afsætter året styrke hjælpen børn pårørende a...,"[afsætte, år, styrke, hjælp, barn, pårørende, ..."
1,nicolai_wammen,socialdemokratiet,2803948786,[dage siden sagde nyt ejendomsvurderingssystem...,"[dag, siden, sige, nytte, ejendomsvurderingssy..."
2,mattias_tesfaye,socialdemokratiet,546254893,[this is literally the same logic many th c am...,"[this, is, literally, the, same, logic, many, ..."
3,jakob_ellemann,venstre,155584627,[tide få fleksibel genåbning vores børn ældre ...,"[tide, få, fleksibel, genåbning, vores, barn, ..."
4,soren_gade,venstre,975064362359623680,[kære marianne synes burde læse lovforslaget i...,"[kære, marianne, syne, burde, læse, lovforslag..."
5,sophie_lohde,venstre,44611200,[flertallet veto dermed røde partier stort set...,"[flertal, veto, dermed, rød, parti, stor, se, ..."
6,lars_lokke,moderaterne,26201346,[mon ikke sjov form argumentation mangler lidt...,"[mon, ikke, sjov, form, argumentation, mangle,..."
7,jacob_mark,sf,2373406198,[slår fast syvtommersøm kom så godt igennem fo...,"[slå, faste, syvtommersøm, komme, så, godte, i..."
8,pia_dyhr,sf,65025162,"[stemmer nok selvom synes gør godt klaus, brug...","[stemme, nok, selvom, syne, gø, godte, klaus, ..."
9,kirsten_andersen,sf,235646319,[arbejde få medarbejdere ser virkeligheden sun...,"[arbejde, få, medarbejder, se, virkelighed, su..."


In [92]:
mini_df = data #data.sample(5, random_state=42)

In [93]:
# mini_df.reset_index(inplace=True, drop=True)
mini_df

,name,party,twitter_id,tweets,tokens
0,magnus_heunicke,socialdemokratiet,22695562,[afsætter året styrke hjælpen børn pårørende a...,"[afsætte, år, styrke, hjælp, barn, pårørende, ..."
1,nicolai_wammen,socialdemokratiet,2803948786,[dage siden sagde nyt ejendomsvurderingssystem...,"[dag, siden, sige, nytte, ejendomsvurderingssy..."
2,mattias_tesfaye,socialdemokratiet,546254893,[this is literally the same logic many th c am...,"[this, is, literally, the, same, logic, many, ..."
3,jakob_ellemann,venstre,155584627,[tide få fleksibel genåbning vores børn ældre ...,"[tide, få, fleksibel, genåbning, vores, barn, ..."
4,soren_gade,venstre,975064362359623680,[kære marianne synes burde læse lovforslaget i...,"[kære, marianne, syne, burde, læse, lovforslag..."
5,sophie_lohde,venstre,44611200,[flertallet veto dermed røde partier stort set...,"[flertal, veto, dermed, rød, parti, stor, se, ..."
6,lars_lokke,moderaterne,26201346,[mon ikke sjov form argumentation mangler lidt...,"[mon, ikke, sjov, form, argumentation, mangle,..."
7,jacob_mark,sf,2373406198,[slår fast syvtommersøm kom så godt igennem fo...,"[slå, faste, syvtommersøm, komme, så, godte, i..."
8,pia_dyhr,sf,65025162,"[stemmer nok selvom synes gør godt klaus, brug...","[stemme, nok, selvom, syne, gø, godte, klaus, ..."
9,kirsten_andersen,sf,235646319,[arbejde få medarbejdere ser virkeligheden sun...,"[arbejde, få, medarbejder, se, virkelighed, su..."


# Starter på at implementere solutions 

## Shingles

In [94]:
def shingle(aString, q, delimiter=' '):
    """
    Input:
        - aString (str): string to split into shingles
        - q (int)
        - delimiter (str): string of the delimiter to consider to split the input string (default: space)
    Return: list of unique shingles
    """
    all_shingles = []
    if delimiter != '':
        words_list = aString.split(delimiter)
    else:
        words_list = aString
    for i in range (len(words_list)-q+1):
        all_shingles.append(delimiter.join(words_list[i:i+q]))
    return list(set(all_shingles))

    
# Example from the Book
# ex_string, q = test, 3
# ex_shingles = shingle(ex_string, q, delimiter='')
# print('Initial string:', ex_string)
# print(f'>> Shingles with q = {q} :',ex_shingles)

# Example from the HINT
for i in range(len(mini_df)):
    ex_string, q = ' '.join(mini_df['tokens'][i]), 2
    ex_shingles = shingle(ex_string, q)
    # assert len(ex_shingles) == 7
    # add shingle to the dataframe
    mini_df.loc[i, 'shingles'] = str(ex_shingles)

    # print('\nInitial string:', ex_string)
    # print(f'>> Shingles with q = {q} :',ex_shingles)

# ex_string, q = 'test', 2
# ex_shingles = shingle(ex_string, q)
# assert len(ex_shingles) == 7
# print('\nInitial string:', ex_string)
# print(f'>> Shingles with q = {q} :',ex_shingles)

Now each shingle is a list of `q` words. We can use the `hash` function to convert each shingle into a number. We can then use the `min` function to find the smallest hash value for each document. This is the signature for the document.

# Lav TF-IDF til en kolonne i dataframen

In [95]:
# According to the book (9.2.2) we can get the n most important (highest tfidf score) words PER document and treat them as a set
# (unlike cosine similarity that treats the vectors with 0s and 1s for each time a word appears in both documents)
#https://www.analyticsvidhya.com/blog/2021/12/how-to-extract-key-phrases-using-tfidf-with-python/

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
vectors = vectorizer.fit_transform(mini_df.tokens.apply(lambda x: " ".join(x)))

In [96]:
vectors

<33x500 sparse matrix of type '<class 'numpy.float64'>'
	with 15400 stored elements in Compressed Sparse Row format>

In [97]:
dict_of_tokens={i[1]:i[0] for i in vectorizer.vocabulary_.items()}

In [98]:
tfidf_vectors = []  # all deoc vectors by tfidf
for row in vectors:
  tfidf_vectors.append({dict_of_tokens[column]:value for (column,value) in zip(row.indices,row.data)})
print("number of documents",len(tfidf_vectors), "the dictionary of document 1", len(tfidf_vectors[3].keys()))

number of documents 33 the dictionary of document 1 491


In [99]:
mini_df['TFIDF_Words'] = [list(tfidf_vectors[i].keys()) for i in range(len(mini_df))]
mini_df['TFIDF_Words'].iloc[0]

['minde',
 'tillykke',
 'mink',
 'simpelthen',
 'samråd',
 'sjov',
 'økonomisk',
 'klima',
 'borgerlig',
 'støjberg',
 'sig',
 'føre',
 'blotte',
 'egentlig',
 'uenig',
 'blå',
 'respekt',
 'ligesom',
 'fra',
 'bolig',
 'miste',
 'trist',
 'afskaffe',
 'forsvar',
 'nato',
 'dyr',
 'politik',
 'handle',
 'økonomi',
 'usa',
 'omstilling',
 'socialdemokrati',
 'mand',
 'kvinde',
 'trække',
 'hvem',
 'landbrug',
 'ellers',
 'regel',
 'gammel',
 'glemme',
 'beslutning',
 'ende',
 'artikel',
 'intet',
 'aarhus',
 'lytte',
 'hjem',
 'alligevel',
 'tvinge',
 'vække',
 'plan',
 'la',
 'skabe',
 'hverken',
 'vist',
 'handler',
 'lægge',
 'grøn',
 'kommission',
 'kritik',
 'endelig',
 'time',
 'tanke',
 'sverige',
 'virksomhed',
 'skole',
 'selv',
 'burde',
 'ven',
 'vildt',
 'høre',
 'arbejdsplads',
 'ren',
 'tale',
 'holdning',
 'amp',
 'også',
 'frihed',
 'fri',
 'rusland',
 'længe',
 'stat',
 'myndighed',
 'opgave',
 'frederiksen',
 'mette',
 'minister',
 'forkert',
 'område',
 'fejl',
 'slå'

# Minhashing algorithm
Now we implement the minhashing algorithm. `minhash` that takes a list of shingles and a seed for the hash function
mapping the shingles, and outputs the minhash

In [100]:
import sys
import os
import mmh3

#hashes a list of strings
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 


#### TODO her tager vi istedet for shingles_list lister af tf-idfs

In [101]:
def minhash(shingles_list, seed):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - seed (int): seed for listhash function
    Return: minhash of given shingles
    """
    minhash_value = None
    for aShingle in shingles_list:
        hashcode = listhash([aShingle], seed)
        if minhash_value == None or hashcode < minhash_value:
            minhash_value = hashcode
    return minhash_value

In [102]:
print(f'MinHash of mini_df.shingles[0]:', minhash(mini_df.shingles[4], 42))

MinHash of mini_df.shingles[0]: -1835151282


In [103]:
def minhash2(shingles_list, k):
    """
    Input:
        - shingles_list (list of str): set of hashes
        - k (int): seed for listhash function
    Return: sequence of k minhashes
    """
    all_minhash = []
    for i in range(k):
        all_minhash.append(minhash(shingles_list, i))
    return all_minhash

In [104]:
k=42
print(f'MinHash of  with k = {k}:\n', minhash2(mini_df.shingles[3], k))

MinHash of  with k = 42:
 [-2128973781, -2079525880, -1962080891, -2099459879, -2119527857, -2122446291, -2127011794, -2138399495, -2135496994, -1918403894, -2066117612, -2145018041, -2067647161, -1978898395, -1901384183, -2129322361, -2122461755, -2131976955, -2118967259, -2002623125, -2069826091, -2141020870, -2139532824, -2140331233, -2145386105, -2034708475, -2028206372, -2056931439, -1822535757, -1985774455, -2018052251, -1914851693, -2140931603, -2069267878, -2050169294, -2135472651, -2107360380, -2073467638, -2122305312, -2125988197, -2136730958, -2110940261]


## Do this for all and save the minhash value to the dataframe 



In [105]:
for i in range(len(mini_df)):
    mini_df.loc[i, 'minhash_tokens'] = minhash(mini_df.tokens[i], 10)

for i in range(len(mini_df)):
    mini_df.loc[i, 'minhas_tf'] = minhash(mini_df.TFIDF_Words[i], 10)

# Signatures

In [106]:
def signature(dict_docs, q = q, num_hashes = k):
    """
    Input:
        - dict_docs (dict of str:str): dictionary of {title:document}
        - q (int)
        - num_hashes (int)
    Return: dictionary consisting of document id’s as keys and signatures as values
    """
    dict_signatures = {}
    total_texts = len(list(dict_docs.keys()))
    counter = 1
    for key,text in dict_docs.items():
        print(f'{counter}/{total_texts} - {key} - Processing...')
        # doc_shingles = shingle(text, q)
        doc_shingles = mini_df.tokens[counter-1]
        # doc_shingles = mini_df.TFIDF_Words[counter-1]
        minhash_values = minhash2(doc_shingles, num_hashes)
        dict_signatures[key] = minhash_values
        counter += 1
    return dict_signatures

In [107]:
# dict_docs is a dictionary of {name:TFIDF_Words}
# dict_docs = {i:j for i,j in zip(mini_df['name'],mini_df['TFIDF_Words'])}
dict_docs = {i:j for i,j in zip(mini_df['name'],mini_df['tokens'])}


signature_dict = signature(dict_docs, q = 2, num_hashes = 100)

1/33 - magnus_heunicke - Processing...
2/33 - nicolai_wammen - Processing...
3/33 - mattias_tesfaye - Processing...
4/33 - jakob_ellemann - Processing...
5/33 - soren_gade - Processing...
6/33 - sophie_lohde - Processing...
7/33 - lars_lokke - Processing...
8/33 - jacob_mark - Processing...
9/33 - pia_dyhr - Processing...
10/33 - kirsten_andersen - Processing...
11/33 - dennis_flydtkjær - Processing...
12/33 - peter_skaarup - Processing...
13/33 - soren_espersen - Processing...
14/33 - alex_vanopslagh - Processing...
15/33 - ole_olesen - Processing...
16/33 - soren_pape - Processing...
17/33 - mette_abildgaard - Processing...
18/33 - rasmus_jarlov - Processing...
19/33 - pelle_dragsted - Processing...
20/33 - mai_villadsen - Processing...
21/33 - rosa_lund - Processing...
22/33 - martin_lidegaard - Processing...
23/33 - samira_nawa - Processing...
24/33 - katrine_robsoe - Processing...
25/33 - pernille_vermund - Processing...
26/33 - lars_mathiesen - Processing...
27/33 - kim_andersen 

In [108]:
# signature_dict

In [109]:
# cosine similarity

def cosine(name1, name2, signature_dict):
    """
    Input:
        - name1 (str): key of the first document S
        - name2 (str): key of the second document T
        - signatures_dict (dict of str:list): dictionary of signatures
    Return: Jaccard similarity between S and T
    """
    signatures_doc1 = np.array(signature_dict[name1])
    signatures_doc2 = np.array(signature_dict[name2])
    # return np.sum(signatures_doc1 == signatures_doc2) #/ len(signatures_doc1)
    return np.dot(signatures_doc1,signatures_doc2)/(norm(signatures_doc1)*norm(signatures_doc2))
    # return len(np.intersect1d(signatures_doc1, signatures_doc2))/len(np.union1d(signatures_doc1, signatures_doc2))


import numpy as np
from numpy.linalg import norm
 
# # define two lists or array
# A = np.array([2,1,2,3,2,9])
# B = np.array([3,4,2,4,5,5])
 
# print("A:", A)
# print("B:", B)
 
# # compute cosine similarity

# print("Cosine Similarity:", cosine)

# Jaccard similarity

In [110]:
def jaccard(name1, name2, signature_dict):
    """
    Input:
        - name1 (str): key of the first document S
        - name2 (str): key of the second document T
        - signatures_dict (dict of str:list): dictionary of signatures
    Return: Jaccard similarity between S and T
    """
    signatures_doc1 = np.array(signature_dict[name1])
    signatures_doc2 = np.array(signature_dict[name2])
    # return np.sum(signatures_doc1 == signatures_doc2) #/ len(signatures_doc1)
    return len(np.intersect1d(signatures_doc1, signatures_doc2))/len(np.union1d(signatures_doc1, signatures_doc2))

In [111]:
# signature_dict['soren_pape']
# signature_dict

In [112]:
first_doc_key = mini_df.name[2]#list(docs.keys())[0]
second_doc_key = mini_df.name[15]#list(docs.keys())[1]
print(f'Jaccard similarity between {first_doc_key} and {second_doc_key}:', jaccard(first_doc_key, second_doc_key, signature_dict))#dict_signatures_docs))

Jaccard similarity between mattias_tesfaye and soren_pape: 0.16279069767441862


# similarity

In [140]:
def similar(signatures_dict, jaccard_threshold=-0.1):
    """
    Input:
        - signatures_dict (dict of str:list): dictionary of signatures
        - jaccard_threshold (float): lower bound for Jaccard similarity score to consider
            two documents as similar
    Return: dictionary of similar items
    """
    list_keys = list(signatures_dict.keys())
    similar_items = {}
    for i in range (len(list_keys)-1):
        for j in range (i+1, len(list_keys)):
            similarity_score = cosine(list_keys[i], list_keys[j], signatures_dict)
            if similarity_score >= jaccard_threshold:
                similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items

In [141]:
found_similar_items = similar(signature_dict)
print('Found similar items:\n', found_similar_items)

Found similar items:
 {('magnus_heunicke', 'nicolai_wammen'): -3.5677843912964958e-12, ('magnus_heunicke', 'mattias_tesfaye'): 2.6018442894373525e-12, ('magnus_heunicke', 'jakob_ellemann'): 9.036028251016883e-13, ('magnus_heunicke', 'soren_gade'): -2.810095442347179e-12, ('magnus_heunicke', 'sophie_lohde'): 1.2177114611478514e-12, ('magnus_heunicke', 'lars_lokke'): -4.600554346390761e-13, ('magnus_heunicke', 'jacob_mark'): -4.1079031132792826e-12, ('magnus_heunicke', 'pia_dyhr'): 1.1070707028355928e-12, ('magnus_heunicke', 'kirsten_andersen'): -8.344428831430693e-13, ('magnus_heunicke', 'dennis_flydtkjær'): 4.205056978620048e-12, ('magnus_heunicke', 'peter_skaarup'): 3.1649582639615297e-12, ('magnus_heunicke', 'soren_espersen'): -3.8887080833385875e-13, ('magnus_heunicke', 'alex_vanopslagh'): -4.099092494600391e-12, ('magnus_heunicke', 'ole_olesen'): 7.711643246737369e-13, ('magnus_heunicke', 'soren_pape'): 2.377727581791308e-12, ('magnus_heunicke', 'mette_abildgaard'): -1.223731482842

In [142]:
# get the most similar items
most_similar_items = sorted(found_similar_items.items(), key=lambda x: x[1], reverse=True)
print('Most similar items:\n', most_similar_items[0:10])

Most similar items:
 [(('peter_skaarup', 'kim_andersen'), 4.653391129588813e-12), (('franciska_rosenkilde', 'torsten_gejl'), 4.634134567040294e-12), (('pelle_dragsted', 'katrine_robsoe'), 4.628364214598911e-12), (('ole_olesen', 'pelle_dragsted'), 4.622879930066708e-12), (('soren_pape', 'franciska_rosenkilde'), 4.618599233075315e-12), (('soren_gade', 'pia_kjarsgaard'), 4.597090026746047e-12), (('dennis_flydtkjær', 'christina_olumeko'), 4.577005208879712e-12), (('mai_villadsen', 'franciska_rosenkilde'), 4.559622014672596e-12), (('mattias_tesfaye', 'soren_espersen'), 4.548153234203276e-12), (('jacob_mark', 'franciska_rosenkilde'), 4.53809632586072e-12)]


In [146]:
# look at the party of the most similar items
similar_items_names = [i for i in found_similar_items.keys()]
for i in range(len(similar_items_names)):
    print(mini_df[mini_df.name.isin(similar_items_names[i])].party)

0    socialdemokratiet
1    socialdemokratiet
Name: party, dtype: object
0    socialdemokratiet
2    socialdemokratiet
Name: party, dtype: object
0    socialdemokratiet
3              venstre
Name: party, dtype: object
0    socialdemokratiet
4              venstre
Name: party, dtype: object
0    socialdemokratiet
5              venstre
Name: party, dtype: object
0    socialdemokratiet
6          moderaterne
Name: party, dtype: object
0    socialdemokratiet
7                   sf
Name: party, dtype: object
0    socialdemokratiet
8                   sf
Name: party, dtype: object
0    socialdemokratiet
9                   sf
Name: party, dtype: object
0        socialdemokratiet
10    danmarksdemokraterne
Name: party, dtype: object
0        socialdemokratiet
11    danmarksdemokraterne
Name: party, dtype: object
0        socialdemokratiet
12    danmarksdemokraterne
Name: party, dtype: object
0     socialdemokratiet
13     liberal_alliance
Name: party, dtype: object
0     socialdemokratiet
1

# LSH

In [133]:
b,r = 20, 5
# assert k == b*r

def lsh(signatures_dict, jaccard_threshold=0.1, seed=42):
    lsh_dict = {}
    for key, values in signatures_dict.items():
        blocks = np.split(np.array(values), b)
        blocks_hash_values = []
        for aBlock in blocks:
            blocks_hash_values.append(mmh3.hash(aBlock, seed))
        lsh_dict[key] = blocks_hash_values
    list_keys = list(lsh_dict.keys())
    similar_items = {}
    for i in range (len(list_keys)-1):
        for j in range (i+1, len(list_keys)):
            common_values = np.intersect1d(lsh_dict[list_keys[i]], lsh_dict[list_keys[j]])
            if len(common_values) > 0:
                # we found a candidate
                similarity_score = jaccard(list_keys[i], list_keys[j], signatures_dict)
                if similarity_score >= jaccard_threshold:
                    similar_items[(list_keys[i], list_keys[j])] = similarity_score
    return similar_items
found_similar_items_with_lsh = lsh(signature_dict)
print('Found similar items with LSH:\n', found_similar_items_with_lsh)

Found similar items with LSH:
 {('pia_dyhr', 'dennis_flydtkjær'): 0.15606936416184972}


In [118]:
# converting a string of text into a vector. Using teh transformer BERT model

# Step one: Use BERT to convert our text into a vector
# Step two:Get the cosine similarity (the cosine of the angle between the two vectors) 
    # of a fixed twitter profiles (vector) and all the other ones
# Step three: Pick the twitter profiles (vectors) with the largest cosine similarity.




In [119]:
# step one
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

text_data = giant_df.CT
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)

NameError: name 'giant_df' is not defined

In [ ]:
X = np.array(giant_df.CT)
# cos_sim_data = pd.DataFrame(cosine_similarity(X))

In [ ]:
# https://towardsdatascience.com/hands-on-content-based-recommender-system-using-python-1d643bf314e4

def give_recommendations(index,print_recommendation = False,print_recommendation_plots= False,print_parties =False):
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:10]
  party_recomm =  giant_df['Party'].loc[index_recomm].values
  result = {'PArty':party_recomm,'Index':index_recomm}
  if print_recommendation==True:
    print('The watched movie is this one: %s \n'%(giant_df['Person'].loc[index]))
    k=1
    for movie in party_recomm:
      print('The number %i recommended movie is this one: %s \n'%(k,movie))
  if print_recommendation_plots==True:
    print('The plot of the watched movie is this one:\n %s \n'%(giant_df['CT'].loc[index]))
    k=1
    for q in range(len(party_recomm)):
      plot_q = giant_df['Overview'].loc[index_recomm[q]]
      print('The plot of the number %i recommended movie is this one:\n %s \n'%(k,plot_q))
      k=k+1
  if print_parties==True:
    print('The party of the twitter profile is this one:\n %s \n'%(giant_df['Party'].loc[index]))
    k=1
    for q in range(len(party_recomm)):
      plot_q = giant_df['Party'].loc[index_recomm[q]]
      print('The plot of the number %i recommended party is this one:\n %s \n'%(k,plot_q))
      k=k+1
  return result



In [ ]:
# check if 'sundhedsminister' is in the list

if 'sundhedsministeren' in doc[0]:
    print('yes')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=30797f9c-952e-45b4-98d4-31c9ac73ae78' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>